In [1]:
!pip install cloudscraper

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import requests 
import pandas as pd
from bs4 import BeautifulSoup as bs
import cloudscraper


otp_url ='http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
otp_form_data = {
  # 'mktId': '060310/3S',
  # 'codeNmisuCd_finder_stkisu0_2': '3S',
  # 'param1isuCd_finder_stkisu0_2': 'ALL',
  'locale': 'ko_KR',
  "share": '1',
  "csvxls_isNo": 'false',
  "name": 'fileDown',
  "url": 'dbms/MDC/STAT/standard/MDCSTAT01701',
  'strtDd': '20130102', # 다운로드 받고싶은 날짜 
  'endDd': '20181230',
  # 'isuCd': 'KR7054620000',
  # 'isuCd': 'KR7265520007',
  # 'isuCd': 'KR7000120006', # 000120/CJ대한통운
  # 'isuCd': 'KR7012030003', # 012030/DB
  'isuCd': 'KR7060310000' # 060310/3S
}
scraper = cloudscraper.create_scraper()
otp = scraper.post(otp_url, params=otp_form_data).text

# otp = requests.post(otp_url, params=otp_form_data).text
otp

'v5GJuDTEOWpud1DHnzAAN/czvUrrTnPokGSWM0X/LEsRtSksuLS7Bnxpl86F7dAOLH4V6xFTl4kR0ZcwnAS+9TgOVRYWf7fkimpo5YvO2zdeDl92bPCV4kU01RYncxbPkc+W4XGqHxCVihg+YOCsbQw9ONExurPWF0lWtSS9NvMh9ZproViOz+JGEuAiYtHcvmKB/MMEoa7R7RA/3KFx01EPNSMBOtzzBS52sZgS6NaH1PnD7WyWSPM9LnXRKGzTRMs95Z8HJAI5WxTKIKigepbUkfIKyoNNMmYDykOlgXM='

In [2]:
csv_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
csv_form_data = requests.post(csv_url, params={'code': otp})
csv_form_data.encoding = 'EUC-KR'

In [3]:
lst_row = []
for row in csv_form_data.text.split('\n'): 
    lst_row.append(row.split(','))
    
df = pd.DataFrame(lst_row[1:], columns=lst_row[0])
df

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,"""2018/12/28""","""2215""","""-25""","""-1.12""","""2270""","""2270""","""2190""","""49523""","""110660610""","""98336869770""","""44395878"""
1,"""2018/12/27""","""2240""","""45""","""2.05""","""2195""","""2275""","""2195""","""99550""","""222378660""","""99446766720""","""44395878"""
2,"""2018/12/26""","""2195""","""45""","""2.09""","""2010""","""2245""","""2005""","""284542""","""595057065""","""97448952210""","""44395878"""
3,"""2018/12/24""","""2150""","""-10""","""-0.46""","""2145""","""2210""","""2135""","""143660""","""311341625""","""95451137700""","""44395878"""
4,"""2018/12/21""","""2160""","""0""","""0.00""","""2170""","""2170""","""2100""","""86462""","""183854135""","""95895096480""","""44395878"""
...,...,...,...,...,...,...,...,...,...,...,...
1468,"""2013/01/08""","""8960""","""1110""","""14.14""","""7920""","""9020""","""7850""","""1878590""","""16305663040""","""365440409600""","""40785760"""
1469,"""2013/01/07""","""7850""","""520""","""7.09""","""7330""","""7940""","""7210""","""745099""","""5701744610""","""320168216000""","""40785760"""
1470,"""2013/01/04""","""7330""","""240""","""3.39""","""7090""","""7330""","""7090""","""357001""","""2573562130""","""298959620800""","""40785760"""
1471,"""2013/01/03""","""7090""","""-60""","""-0.84""","""7180""","""7270""","""7070""","""226940""","""1623171010""","""289171038400""","""40785760"""


In [4]:
dic_market2code = {}
stocks = pd.read_csv('stocks.csv', encoding = 'EUC-KR')
for market, data in stocks.groupby('시장구분'): 
    dic_market2code[market] = data['표준코드'].values
dic_market2code

{'KONEX': array(['KR7260870001', 'KR7183410000', 'KR7076340009', 'KR7243870003',
        'KR7244880001', 'KR7288490006', 'KR7257990002', 'KR7317860005',
        'KR7229500004', 'KR7323350009', 'KR7271850000', 'KR7178600003',
        'KR7114920002', 'KR7302920004', 'KR7348840000', 'KR7199150004',
        'KR7227420007', 'KR7176750008', 'KR7163430002', 'KR7285770004',
        'KR7403360001', 'KR7092590009', 'KR7179720008', 'KR7238500003',
        'KR7223220005', 'KR7162120000', 'KR7253610000', 'KR7200580009',
        'KR7233250000', 'KR7236340006', 'KR7267060002', 'KR7135160000',
        'KR7322970005', 'KR7208890004', 'KR7354390007', 'KR7281310003',
        'KR7266470004', 'KR7199290008', 'KR7322190000', 'KR7206950008',
        'KR7191600006', 'KR7215050006', 'KR7086220001', 'KR7210120002',
        'KR7266170000', 'KR7258250000', 'KR7258540004', 'KR7232830000',
        'KR7217320001', 'KR7236030003', 'KR7103660007', 'KR7245450002',
        'KR7286000005', 'KR7149010001', 'KR7149300006',

In [5]:
def krx_download(date:list, markets:list):
    import requests 
    import pandas as pd
    from bs4 import BeautifulSoup as bs
    from tqdm import tqdm
    
    for market in markets: # 마켓 별 csv 파일 저장 
        df_market = pd.DataFrame()
        lst_code = dic_market2code[market]
        for code in tqdm(lst_code): # 코드 별 데이터프레임 저장 
            otp_url ='http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
            otp_form_data = {
              'locale': 'ko_KR',
              "share": '1',
              "csvxls_isNo": 'false',
              "name": 'fileDown',
              "url": 'dbms/MDC/STAT/standard/MDCSTAT01701',
              'strtDd': date[0], # 다운로드 받고싶은 날짜 
              'endDd': date[1],
              'isuCd': code # 060310/3S
            }

            otp = requests.post(otp_url, params=otp_form_data).text
            
            csv_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
            csv_form_data = requests.post(csv_url, params={'code': otp})
            csv_form_data.encoding = 'EUC-KR'

            lst_row = []
            for row in csv_form_data.text.split('\n'): 
                lst_row.append(row.split(','))

            df = pd.DataFrame(lst_row[1:], columns=lst_row[0])
            df['Code'] = code 
            df_market = df_market.append(df)
            

            df_market.to_csv(f'{market}_{date[0]}_{date[1]}.csv', index=None)

In [8]:
import warnings
warnings.filterwarnings('ignore')
krx_download(['20130102', '20181230'], ['KOSDAQ'])

100%|█████████████████████████████████████| 1603/1603 [1:19:27<00:00,  2.97s/it]


In [7]:
df = pd.read_csv('KOSDAQ_20130102_20181230.csv');df

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,Code
0,"""2018/12/28""","""12450""","""150""","""1.22""","""12600""","""12600""","""11900""","""6682""","""80856300""","""59232132450""","""4757601""",KR7260870001
1,"""2018/12/27""","""12300""","""-100""","""-0.81""","""12750""","""12750""","""11900""","""5100""","""61817350""","""58518492300""","""4757601""",KR7260870001
2,"""2018/12/26""","""12400""","""400""","""3.33""","""12400""","""12400""","""11850""","""2841""","""35025950""","""58994252400""","""4757601""",KR7260870001
3,"""2018/12/24""","""12000""","""-300""","""-2.44""","""11800""","""12600""","""11800""","""3560""","""42838000""","""57091212000""","""4757601""",KR7260870001
4,"""2018/12/21""","""12300""","""200""","""1.65""","""12700""","""12700""","""11600""","""4788""","""58120700""","""58518492300""","""4757601""",KR7260870001
...,...,...,...,...,...,...,...,...,...,...,...,...
59841,"""2016/08/01""","""20000""","""650""","""3.36""","""20000""","""20000""","""20000""","""2250""","""45000000""","""40306800000""","""2015340""",KR7212310007
59842,"""2016/07/29""","""19350""","""-650""","""-3.25""","""19900""","""19900""","""19350""","""1501""","""29869350""","""38996829000""","""2015340""",KR7212310007
59843,"""2016/07/28""","""20000""","""100""","""0.50""","""19900""","""20000""","""19900""","""1171""","""23370000""","""40306800000""","""2015340""",KR7212310007
59844,"""2016/07/27""","""19900""","""0""","""0.00""","""0""","""0""","""0""","""0""","""0""","""40105266000""","""2015340""",KR7212310007


In [20]:
# .describe() 함수를 이용하여 요약 통게량을 출력합니다.
df.describe()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
count,1199780,1199780,1199780,1199780,1199780,1199780,1199780,1199780,1199780,1199780,1199780
unique,1473,6735,1780,4283,6740,6795,6704,449404,1173044,431158,3859
top,"""2018/12/28""","""12000""","""0""","""0.00""","""0""","""0""","""0""","""0""","""0""","""23000000000""","""10000000"""
freq,873,1528,99977,99977,10398,10398,10398,10379,10379,338,11629


깃헙 업로드
- 마켓별로 업로드
- 컬럼: Code, 일자, 종가, 시가,  고가, 저가, 거래량 , 등락률
- 컬럼명: Code, Date, Close, Open, High, Low, Volume, Change
- 데이터타입 변경 (주가, 거래대금: int, 등락률: float)

In [15]:
df['Code'] = 'KR7060310000'

In [13]:
def krx_download(date:list, markets:list):
    import requests 
    import pandas as pd
    from bs4 import BeautifulSoup as bs
    from tqdm import tqdm
    import cloudscraper

    
    df_total = pd.DataFrame()
    for market in markets: # 마켓 별 csv 파일 저장 
        df_market = pd.DataFrame()
        lst_code = dic_market2code[market]
        for code in tqdm(lst_code): # 코드 별 데이터프레임 저장 
            otp_url ='http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
            otp_form_data = {
              'locale': 'ko_KR',
              "share": '1',
              "csvxls_isNo": 'false',
              "name": 'fileDown',
              "url": 'dbms/MDC/STAT/standard/MDCSTAT01701',
              'strtDd': date[0], # 다운로드 받고싶은 날짜 
              'endDd': date[1],
              'isuCd': code # 060310/3S
            }
            
            scraper = cloudscraper.create_scraper()
            otp = scraper.post(otp_url, params=otp_form_data).text
            
            csv_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
            csv_form_data = scraper.post(csv_url, params={'code': otp})
            csv_form_data.encoding = 'EUC-KR'

            lst_row = []
            for row in csv_form_data.text.split('\n'): 
                lst_row.append(row.split(','))

            df = pd.DataFrame(lst_row[1:], columns=lst_row[0])
            df['Code'] = code 
            df_market = df_market.append(df)
        
        df_market['market'] = market
        df_total = df_total.append(df_market)
            

    df_total.to_csv(f'{date[0]}_{date[1]}.csv', index=None)

# 해당 url에서 데이터셋 다운받는 과정 

### 종목 선택하는
<a target="_blank" href="/equities/apple-computer-inc" title="Apple Inc">Apple</a>

### 시장 바꾸는 
<button class="inv-button outlined relative-selector_relative-selector-button__sIOFX" type="button" data-test="relative-selector"><div class="relative-selector_flag-icon__O_3BE"><div class="lazyload-wrapper "><span data-test="flag-US" class="flag_flag__Vuf36 flag_flag--US__ycADH align-middle" role="img"></span></div></div><span class="text-xs ml-1">NASDAQ</span><svg viewBox="0 0 24 24" width="1em" fill="none" class="ml-1 text-3xs" style="height: auto;"><path d="M22 7L12.5 16.5L3 7" stroke="currentColor" stroke-width="3"></path></svg></button>

### historical data
<div class="navbar_selected-non-link-sub__Eb0po"> Historical Data</div>

### 날짜 바꾸는
<div class="DatePickerWrapper_input-text__PDRoD DatePickerWrapper_center__zPpJy">10/20/2022 - 11/20/2022</div>

### 다운로드하는
<div class="download-data_download-data__jxNYT"><svg viewBox="0 0 24 24" width="1em" fill="none" class="download-data_icon__WeWYx" style="height: auto;"><path fill-rule="evenodd" clip-rule="evenodd" d="M17.4151 9.59979L14.0672 9.59979L14.0672 2L11.0672 2L11.0672 9.59979L8.13122 9.59979L12.7732 16.1907L17.4151 9.59979ZM1.4918 16.1907L1.4918 22.4926L1.4918 23.9926H2.9918L22.1095 23.9926H23.6095V22.4926V16.1907H20.6095V20.9926L4.4918 20.9926L4.4918 16.1907H1.4918Z" fill="currentColor"></path></svg><a download="NVDA Historical Data.csv" target="_self" href="blob:https://www.investing.com/812c4134-2b89-4e78-8f48-c347a2003606"><span class="download-data_text__Myrn3">Download Data</span></a></div>



# 종목 url 찾는과정
- 국가 지정(선택)
- 일단 목록에서 모든 종목 url을 찾는다.
- 페이지를 넘기며 반복한다.


## 국가 지정
<td class="flag"><span class="ceFlags USA"></span></td>
<td class="flag"><span class="ceFlags USA"></span></td>


## 목록에서 모든 종목 url을 찾는다.

### 종목코드로 tag 위치 찾기
<td data-column-name="viewData.symbol" class="left">ANZBY</td>

### 찾은 tag에서 url 복사해오기
<a target="_blank" href="/equities/au---nz-banking-group-ltd" title="Australia and New Zealand Banking Group Ltd ADR">ANZ Banking Group ADR</a>


## 페이지를 넘기며 반복한다.

### 페이지 넘어가기
<a href="javascript:void(0);" onclick="stockScreenerPaginationClick(2)" title="Show results 51 to 100 of 12111" class="pagination">2</a>

<a href="javascript:void(0);" onclick="stockScreenerPaginationClick(3)" title="Show results 101 to 150 of 12111" class="pagination">3</a>

<a href="javascript:void(0);" onclick="stockScreenerPaginationClick(3)" title="Show results 101 to 150 of 12111" class="pagination">3</a>